In [71]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import InvalidSelectorException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.by import By
import time
print('Modules imported.')
# Setting Webdriver
DRIVER_PATH = r'C:\Users\User\Desktop\chromedriver.exe'  # BILGISAYARDA chromedriver'ın bulunduğu yer
options = Options() 
options.headless = False
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
driver.get("https://sim.csb.gov.tr/STN/STN_Report/StationDataDownloadNew")
from selenium.webdriver.support.ui import Select


Modules imported.


In [64]:
## çekilemeyen sayfaları looptan sonra çıkartacak olan DataFrame
hata_frame =pd.DataFrame()

In [ ]:
##### Key for istasyon
driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/form/fieldset[1]/div[1]/div[1]/div[1]/div[5]/div/div/span[1]/span/span[2]').click()
time.sleep(3)
key_istasyon_liste = driver.find_elements_by_xpath('/html/body/div[12]/div/div[3]/ul/li')
a = len(key_istasyon_liste)
driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/form/fieldset[1]/div[1]/div[1]/div[1]/div[5]/div/div/span[1]/span/span[2]').click()


writer = pd.ExcelWriter(r'C:\Users\User\Desktop\datum_2.xlsx', engine='xlsxwriter')
for j in range(376,a): 
    try:
        #############################
        # Sayfa üzerindeki işlemler #
        #############################
        # Tüm illerin seçilmesi
        WebDriverWait(driver, 5).until( EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[1]/div[1]/div[1]/div[1]/div[2]/div/div/div/div/span[3]'))).click()
        # Günlük verilerin seçilmesi
        WebDriverWait(driver, 5).until( EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[1]/div[1]/div[1]/div[2]/div[1]/div/div/label[2]'))).click() 
        # Tarihin seçilmesi
        WebDriverWait(driver, 5).until( EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[1]/div[1]/div[1]/div[2]/div[2]/div/div/span[1]/span'))).click() 
        # Tarih kısmının silinmesi
        WebDriverWait(driver, 5).until( EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[1]/div[1]/div[1]/div[2]/div[2]/div/div/span[1]/span/input'))).clear() 
        # Tarih kısmına ilk tarihin atılması
        WebDriverWait(driver, 5).until( EC.element_to_be_clickable((By.XPATH, '//*[@id="StationDataDownload_StartDateTime"]'))).send_keys("01.01.2005")
        #İstasyon tıklama
        time.sleep(1)
        # İstasyon isimlerinin görülmesi için Menüye tıklama
        driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/form/fieldset[1]/div[1]/div[1]/div[1]/div[5]/div/div/span[1]/span/span[2]').click()
        time.sleep(3)
        
        # istasyon isimleri
        key_istasyon_liste = driver.find_elements_by_xpath('/html/body/div[12]/div/div[3]/ul/li')
        print(key_istasyon_liste[j].text,j)
        # istasyon seçme
        key_istasyon_liste[j].click()
        #Sorgulama
        WebDriverWait(driver, 5).until( EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[1]/div[1]/div[2]/div[1]/div/div/div/button'))).click()
        # yüklenme ekranının beklenmesi
        WebDriverWait(driver, 20).until_not(EC.presence_of_element_located((By.CLASS_NAME, "jquery-loading-modal")))
        
        
        # Veri varsa
        if len(driver.find_elements_by_class_name("toast-top-right")) == 0:
            time.sleep(1)
            # menüye mevcut mu kontrolü
            WebDriverWait(driver,20).until( EC.visibility_of_all_elements_located((By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[3]/div/div[5]/span[1]')))
            time.sleep(1)
            # menüye tıklama
            WebDriverWait(driver,20).until( EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[3]/div/div[5]/span[1]'))).click()
            time.sleep(1)
            # tümü yazısına sahip seçeneği bulma
            buttons =driver.find_elements_by_xpath(("//ul/li[text()='Tümü']"))
            time.sleep(1)
            button = []
            # birden fazla "Tümü" çıkması halinde seçeneği teke indirme
            for i in range(0,len(buttons)):
                if buttons[i].text == 'Tümü':
                    button = buttons[i]
                else:
                    continue
            while True:
                try:
                    if len(button) > 1:
                        button = button[0]
                except TypeError:
                    break
            # tümünü seçme    
            button.click()
        # veri yoksa
        else:
            # çıkan hatayı kapatma
            driver.find_elements_by_class_name("toast-close-button")[0].click() 
            # sayfayı temizleme
            WebDriverWait(driver , 5).until( EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[1]/div[1]/div[2]/div[2]/div/div/div/button'))).click()
            # devam etme
            continue 

        ######################################################
                        # Veri işlemleri #
        ######################################################
        #Tarihler
        df_1 = pd.read_html(driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[3]/div/div[3]').get_attribute('outerHTML'))[0]
        # Veriler
        df_2 = pd.read_html(driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[3]/div/div[4]').get_attribute('outerHTML'))[0]
        # sütun isimleri
        a=[]
        for i in range(0,len(df_2.columns)):
            a.append(pd.read_html(driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/form/fieldset[3]/div/div[2]/div[2]").get_attribute('outerHTML'))[0].columns[i][1])
        df_2.columns =a
        # Veri setini oluştuma
        datum = pd.DataFrame()
        datum["Tarih"] = df_1
        for i in range(0, len(df_2.columns)):
            datum[df_2.columns[i]] = df_2[df_2.columns[i]]
        datum.insert(0,"istasyon",pd.read_html(driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/form/fieldset[3]/div/div[2]/div[2]").get_attribute('outerHTML'))[0].columns[0][0])
        ## Sayfa Adı uzunluğu kısaltıcı
        if len(pd.read_html(driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/form/fieldset[3]/div/div[2]/div[2]").get_attribute('outerHTML'))[0].columns[0][0]) >30 :
            sheet_name = pd.read_html(driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/form/fieldset[3]/div/div[2]/div[2]").get_attribute('outerHTML'))[0].columns[0][0][:29]
        else:
            sheet_name = pd.read_html(driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/form/fieldset[3]/div/div[2]/div[2]").get_attribute('outerHTML'))[0].columns[0][0]
        # veriyi excele yazdırma
        datum.to_excel(writer, sheet_name=sheet_name, index=False)
    
        ######################################################
                        # Veri işlemleri #
        ######################################################

        #Temizle
        WebDriverWait(driver,5).until( EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[1]/div[1]/div[2]/div[2]/div/div/div/button'))).click()
        time.sleep(2)

        # kodun kasmaması için 15  aramada bir driver'ın açılıp kapanması
        if j ==0:
            continue
        elif j//15 - j/15 == 0:
            driver.quit()
            driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
            driver.get("https://sim.csb.gov.tr/STN/STN_Report/StationDataDownloadNew")
            time.sleep(2)
        else:
            continue
    # Hata halinde hata koduyla beraber çekilmeyen numaranın kaydedilmesi
    except TimeoutException:
        hata = pd.DataFrame([pd.read_html(driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/form/fieldset[3]/div/div[2]/div[2]").get_attribute('outerHTML'))[0].columns[0][0], "TimeoutException"]).T
        hata_frame = hata_frame.append(hata)
        WebDriverWait(driver,5).until( EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[1]/div[1]/div[2]/div[2]/div/div/div/button'))).click()
        time.sleep(2)
        continue
    except InvalidSelectorException:
        hata =  pd.DataFrame([pd.read_html(driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/form/fieldset[3]/div/div[2]/div[2]").get_attribute('outerHTML'))[0].columns[0][0], "InvalidSelectorException"]).T
        hata_frame = hata_frame.append(hata)
        WebDriverWait(driver,5).until( EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[1]/div[1]/div[2]/div[2]/div/div/div/button'))).click()
        time.sleep(2)
        continue
    except ElementClickInterceptedException:
        hata =  pd.DataFrame([pd.read_html(driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/form/fieldset[3]/div/div[2]/div[2]").get_attribute('outerHTML'))[0].columns[0][0], "ElementClickInterceptedException"]).T
        hata_frame = hata_frame.append(hata)
        WebDriverWait(driver,5).until( EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[1]/div[1]/div[2]/div[2]/div/div/div/button'))).click()
        time.sleep(2)
        continue
    except ElementNotInteractableException:
        hata =  pd.DataFrame([pd.read_html(driver.find_element(By.XPATH, "/html/body/div[2]/div/div[2]/form/fieldset[3]/div/div[2]/div[2]").get_attribute('outerHTML'))[0].columns[0][0], "ElementNotInteractableException"]).T
        hata_frame = hata_frame.append(hata)
        WebDriverWait(driver,5).until( EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div[2]/form/fieldset[1]/div[1]/div[2]/div[2]/div/div/div/button'))).click()
        time.sleep(2)
        continue

    
                

writer.save()
writer.close()
hata_frame



In [70]:
## Hata halinde kayıt için kodu çalıştırın. Ve bir önceki Kod bloğunu yeniden çalıştırın.
writer.save()
writer.close()

In [90]:
datum = pd.read_excel(r'C:\Users\User\Desktop\datum.xlsx', sheet_name = None)


dict_keys(['Yalova', 'Yalova - Altınova-MTHM', 'Yalova - Armutlu-MTHM', 'Yozgat', 'Yozgat - Sorgun', 'Zonguldak - Çatalağzı Cumayanı', 'Zonguldak - Çatalağzı Kuzyaka', 'Zonguldak - Çaycuma', 'Zonguldak - Karadeniz Ereğli', 'Zonguldak - Kilimli', 'Zonguldak - Kozlu', 'Zonguldak - Muslu Tepeköy', 'Zonguldak - Trafik'])

In [104]:
datum_key = list()
for i in datum.keys():
    datum_key.append(i)

'Adana - Çatalan'

In [109]:
datum_finale = pd.DataFrame()
datum = pd.read_excel(r'C:\Users\User\Desktop\datum.xlsx', sheet_name = None)
datum_key = list()
for i in datum.keys():
    datum_key.append(i)

for i in range(0,len(datum_key)):
    vars()["df_"+datum_key[i]] = pd.read_excel(r'C:\Users\User\Desktop\datum.xlsx', sheet_name = datum_key[i])
    
for i in range(0,len(datum_key)):
    datum_finale = datum_finale.append(vars()["df_"+datum_key[i]], ignore_index=False)

In [111]:
datum_finale = pd.DataFrame()
datum = pd.read_excel(r'C:\Users\User\Desktop\datum_1.xlsx', sheet_name = None)
datum_key = list()
for i in datum.keys():
    datum_key.append(i)

for i in range(0,len(datum_key)):
    vars()["df_"+datum_key[i]] = pd.read_excel(r'C:\Users\User\Desktop\datum_1.xlsx', sheet_name = datum_key[i])
    
for i in range(0,len(datum_key)):
    datum_finale = datum_finale.append(vars()["df_"+datum_key[i]], ignore_index=False)

In [112]:
datum_finale = pd.DataFrame()
datum = pd.read_excel(r'C:\Users\User\Desktop\datum_2.xlsx', sheet_name = None)
datum_key = list()
for i in datum.keys():
    datum_key.append(i)

for i in range(0,len(datum_key)):
    vars()["df_"+datum_key[i]] = pd.read_excel(r'C:\Users\User\Desktop\datum_2.xlsx', sheet_name = datum_key[i])
    
for i in range(0,len(datum_key)):
    datum_finale = datum_finale.append(vars()["df_"+datum_key[i]], ignore_index=False)

In [113]:
datum_finale

,istasyon,Tarih,PM10 ( µg/m3 ),SO2 ( µg/m3 ),NO2 ( µg/m3 ),NOX ( µg/m3 ),O3 ( µg/m3 ),CO ( µg/m3 )
0,Yalova,24.02.2007,-,-,-,-,-,NaN
1,Yalova,25.02.2007,3758,162,-,-,-,NaN
2,Yalova,26.02.2007,9151,175,-,-,-,NaN
3,Yalova,27.02.2007,12363,373,-,-,-,NaN
4,Yalova,28.02.2007,7751,399,-,-,-,NaN
...,...,...,...,...,...,...,...,...
6054,Zonguldak - Trafik,02.10.2021,2683,992,3171,5624,NaN,39053
6055,Zonguldak - Trafik,03.10.2021,2497,965,3054,4763,NaN,30537
6056,Zonguldak - Trafik,04.10.2021,3756,974,4105,7907,NaN,41569
6057,Zonguldak - Trafik,05.10.2021,3094,1016,4041,7603,NaN,39295
